Link Google Colab : https://colab.research.google.com/drive/1R5l8Q1rnYRLpujrRUCysVKtNsXEhK7pm?usp=sharing

# **TUGAS BYTE PAIR ENCODING**

NAMA ANGGOTA

1. Karunia Perjuangan M ( 20/456368/TK/50498 )

2. Pramudya Kusuma H  ( 20/460558/TK/51147 )

Untuk mengambil dataset yang ada di drive, kita perlu melakukan mount dahulu ke folder drive kita.

# Mengambil Corpus


Sebelum membuat tokenizer, kita perlu menentukan dahulu korpus yang akan digunakan. Kita akan menggunakan corpus berbahasa Indonesia. Corpus yang kita gunakan bersumber dari Leipzig Corpora yang berisi 10000 baris kalimat berita yang ada pada tahun 2016 dan seluruhnya berbahasa Indonesia.

Dataset asli bisa diambil dari https://wortschatz.uni-leipzig.de/en/download/Indonesian .

In [ ]:
with open("./ind_newscrawl_2016_10K-sentences.txt","r") as fp:
  file_text = fp.read()


Catatan : Jika kode di atas tidak bekerja, tambahkan shortcut folder https://drive.google.com/drive/folders/1NlfSIo0--8hCofH3OLjclkAOvAfTaQlK?usp=sharing ke My Drive dan sesuaikan path di atas.

Karena format dataset adalah Tab Separated Value dalam bentuk .txt, kita perlu memodifikasi seperlunya agar data menjadi list berisi kumpulan kalimat.

In [ ]:
list_line = file_text.split("\n")
print(list_line[-4:])
list_line.pop()
print("Jumlah Kalimat : ",len(list_line))

['9998\tZulkifli Hasan mengatakan, Indonesia adalah negara yang terdiri dari beragam suku, agama, budaya, dan bahasa.', '9999\tZulkifli Hasan tekankan RUU Pemilu utamakan hak demokrasi rakyat25 Oktober 2016 15:12 Ketua MPR RI Zulkifli Hasan menekankan agar Rancangan Undang Undang tentang Penyelenggaraan Pemilihan Umum mengutamakan hak demokrasi rakyat."', '10000\tZwensi Areros sendiri harus ber-saing dengan 37 finalis lainnya dari 33 propinsi se-Indonesia.', '']
Jumlah Kalimat :  10000


In [ ]:
list_sentence = []
for line in list_line:
  list_sentence.append(line.split('\t')[-1])

print(list_sentence[-10:])

['Zaenal kembali mengangkat isu bahwa SBY pernah menikah sebelum masuk Akademi Militer (Akmil).', 'Zat yang benar-benar anda perlukan untuk sistem kekebalan tubuh yang sehat, mineral ini juga mendukung penyembuhan luka ringan.', 'ZenFone 3 Max mengadopsi konsep inti dari desain industrial, yakni menggabungkan estetika keindahan dengan build quality yang tinggi untuk menghasilkan pengalaman pengguna yang memuaskan.', 'Zidane mengatakan tidak akan mendesak Ronaldo untuk segera kembali merumput.', 'Zidane sendiri disebut bintang Madrid, Cristiano Ronaldo, sebagai kunci sukses Los Blancos.', 'Ziyad mengungkapkan sebagian besar pengungsi berasal dari Daraya karena wilayah itu dekat dengan Judaidah Artus.', 'Zulkifli Berharap agar generasi muda Indonesia mampu memetik pelajaran dari beberapa persitiwa panas jelang pilkada DKI Jakarta yakni harus terus menjaga persatuan dan kesatuan bangsa yang berbhinneka apapun yang terjadi.', 'Zulkifli Hasan mengatakan, Indonesia adalah negara yang terdiri

Output di atas adalah 10 kalimat terakhir yang muncul di dataset.

# Algoritma BPE

Kode untuk algoritma Byte Pair Encoding didasarkan pada tutorial yang ada di https://huggingface.co/course/chapter6/5 dengan beberapa perubahan seperlunya.

Hal pertama yang perlu kita lakukan adalah menghitung frekuensi kata yang muncul dalam corpus. Karena kita ingin membuat algoritma secara sederhana saja, preprocessing yang kita lakukan sebelum menghitung frekuensi kata adalah memisah kata per kalimat dengan spasi dan me-*lower case*-kan semua huruf.

Dengan demikian, kita akan mendapatkan dictionary `word_freqs` yang berisi pasangan kata jumlah kemunculannya dalam dataset.

In [ ]:
word_freqs = {}

for text in list_sentence:
    words = text.lower().split(" ")
    for word in words:
      if word_freqs.get(word) == None:
        word_freqs[word] = 1
      else:
        word_freqs[word] +=1

print(word_freqs)

{'10.pemberian': 1, 'label': 4, 'yang': 4123, 'berisi': 13, 'keterangan:': 1, 'tanggal': 33, 'peng-ambilan,': 1, 'kode': 8, 'pengambilan': 5, '(nama': 2, 'pengambil),': 1, 'nomor': 63, 'con-toh': 3, 'tanah,': 3, 'lokasi': 72, 'lokasi,': 5, 'desa,': 8, 'kecamatan,': 2, 'kabupaten)': 1, 'dan': 3035, 'kedalaman': 3, 'contoh.': 1, '11.': 1, 'pada': 791, '6': 25, 'desember': 69, '1999,': 1, 'presiden': 92, 'saat': 504, 'itu,': 374, 'abdurrahman': 5, 'wahid': 6, 'alias': 28, 'gus': 4, 'dur,': 1, 'mengusut': 3, 'kembali': 132, 'kekayaan': 16, 'soeharto': 3, 'menunjuk': 8, 'jaksa': 26, 'agungnya,': 1, 'marzuki': 2, 'darusman,': 1, 'memulai': 9, 'penyidikan': 5, 'lagi': 127, 'atas': 148, 'soeharto.': 1, '“12': 1, 'pekan': 66, 'pertama': 105, 'kehamilan': 3, 'itu': 978, 'sangat': 213, 'melelahkan.': 1, '1': 74, 'december': 2, '2016,': 30, '05:47': 1, 'wib': 55, 'rakyat': 50, 'bertekad': 4, 'jaga': 5, 'kebinekaan': 2, 'dengan': 1435, 'mengenakan': 14, 'ikat': 3, 'kepala': 177, 'merah': 20, 'putih

Kita membuat variabel baru untuk memecah setiap kata menjadi huruf-huruf yang belum di-merge. 

In [ ]:
splits = {word: [char for char in word] for word in word_freqs.keys()}

In [ ]:
print(splits)

{'10.pemberian': ['1', '0', '.', 'p', 'e', 'm', 'b', 'e', 'r', 'i', 'a', 'n'], 'label': ['l', 'a', 'b', 'e', 'l'], 'yang': ['y', 'a', 'n', 'g'], 'berisi': ['b', 'e', 'r', 'i', 's', 'i'], 'keterangan:': ['k', 'e', 't', 'e', 'r', 'a', 'n', 'g', 'a', 'n', ':'], 'tanggal': ['t', 'a', 'n', 'g', 'g', 'a', 'l'], 'peng-ambilan,': ['p', 'e', 'n', 'g', '-', 'a', 'm', 'b', 'i', 'l', 'a', 'n', ','], 'kode': ['k', 'o', 'd', 'e'], 'pengambilan': ['p', 'e', 'n', 'g', 'a', 'm', 'b', 'i', 'l', 'a', 'n'], '(nama': ['(', 'n', 'a', 'm', 'a'], 'pengambil),': ['p', 'e', 'n', 'g', 'a', 'm', 'b', 'i', 'l', ')', ','], 'nomor': ['n', 'o', 'm', 'o', 'r'], 'con-toh': ['c', 'o', 'n', '-', 't', 'o', 'h'], 'tanah,': ['t', 'a', 'n', 'a', 'h', ','], 'lokasi': ['l', 'o', 'k', 'a', 's', 'i'], 'lokasi,': ['l', 'o', 'k', 'a', 's', 'i', ','], 'desa,': ['d', 'e', 's', 'a', ','], 'kecamatan,': ['k', 'e', 'c', 'a', 'm', 'a', 't', 'a', 'n', ','], 'kabupaten)': ['k', 'a', 'b', 'u', 'p', 'a', 't', 'e', 'n', ')'], 'dan': ['d', 'a

Lalu kita membuat fungsi `compute_pair_freqs` yang menghitung frekuensi atau jumlah kemunculan pasangan token berjejer (artinya di awal training, kita menghitung frekuensi jumlah pasangan dua huruf berjejeran terurut).

In [ ]:
def compute_pair_freqs(splits):
    pair_freqs = {}
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            if pair_freqs.get(pair) == None:
              pair_freqs[pair] = freq
            else:
              pair_freqs[pair] += freq
    return pair_freqs

Untuk pertama kali, kita hitung jumlah pasangan huruf berjejer pada `splits`, lalu kita tampilkan 5 pasangan pertama yang ada sebagai contoh.

In [ ]:
pair_freqs = compute_pair_freqs(splits)

In [ ]:
for i, key in enumerate(pair_freqs.keys()):
    print(f"{key}: {pair_freqs[key]}")
    if i >= 5:
        break

('1', '0'): 252
('0', '.'): 81
('.', 'p'): 2
('p', 'e'): 9448
('e', 'm'): 7766
('m', 'b'): 3362


Lalu kita melakukan *looping* untuk mencari pasangan huruf yang paling banyak ditemukan. Tujuan dari hal ini adalah menentukan pasangan token yang akan digabungkan.

In [ ]:
best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

('a', 'n') 46547


Penggabungan ini kita lakukan dalam fungsi `merge_pair` Fungsi ini akan menggabungkan token `a` dan `b` dalam setiap kemunculan dalam `splits`

In [ ]:
def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits

Lalu kita akan membuat sebuah list `vocab` yang berisi token-token yang ingin dibuat. Untuk awalnya, token-token ini terdiri dari setiap karakter yang ada dalam dataset.

In [ ]:
vocab = []

for word in word_freqs.keys():
    for letter in word:
        if letter not in vocab:
            vocab.append(letter)
vocab.sort()

print(vocab)

['!', '"', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', '@', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '~', '«', '\xad', '»', 'á', 'â', 'ã', 'è', 'é', 'ü', 'ć', '˜', 'α', 'β', '\u200e', '\u200f', '–', '—', '‘', '’', '“', '”', '…', '\u202a', '€', '™', '\ue00f']


Lalu pada bagian utama algoritma BPE ini, kita akan melakukan loop untuk mempelajari semua token yang diinginkan dengan batas tertentu.

Pada kasus ini, kita mengambil batas 5000 token agar tidak terlalu banyak dan tidak terlalu sedikit pula.

Inti dari proses *looping* ini adalah mencari pasangan token berdekatan dengan frekuensi terbesar, lalu menggabungkan kedua token tersebut ke dalam token baru. Perlu diingat bahwa token yang sudah ada tidak terhapus.

Dalam proses ini, kita juga menggunakan dictionary `merges` yang berisi daftar token yang akan di-*merge* serta hasil *merge*-nya untuk memudahkan tokenization nantinya

In [ ]:
vocab_size = 5000
merges = {}
while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])



Untuk jaga-jaga, kita simpan list vocabulary ke dalam sebuah file `.txt`.

In [ ]:
with open("./list_vocab.txt",'w') as vocab_file:
  for vocabulary in vocab:
    vocab_file.write(vocabulary+"\n")
  

Lalu kita buat fungsi untuk men-tokenize teks baru, pertama dilakukan pre tokenize dengan melakukan proses *lower case* dan pemisahan berdasar spasi, lalu menerapkan merge perlahan-lahan dengan daftar merge yang telah di-*learn* sebelumnya dalam variabel `merges`.

In [ ]:
def tokenize(text):
    pre_tokenized_text = text.lower().split(" ")
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

# Testing

Mari kita uji ke dalam beberapa kalimat. Kita buat beberapa kasus:

1. 4 kalimat pertama merupakan kalimat berbahasa Indonesia seperti dataset.
2. Kalimt kelima menggunakan huruf latin seperti dataset, namun dengan bahasa Inggris yang jauh berbeda dengan bahasa Indonesia.
3. Kalimat keenam berbahasa Indonesia, namun dengan campuran karakter asing yang tidak ada dalam *vocab*.
4. Kalimat terakhir tidak berbahasa Indonesia dan menggunakan karakter non latin.

In [ ]:
text_examples = [
    "Mantan Kadiv Propam Polri Irjen Ferdy Sambo resmi diberhentiakan secara tidak hormat sebagai Anggota Polri.",
    "Real Madrid berhasil menjadi kampiun Liga Champions 2021-2022 setelah menyudahi perlawanan sengit dari Liverpool.",
    "Pasar Kangen merupakan juga ajang kreativitas dan produktivitas dalam pengolahan pangan berbasis nilai-nilai local dan bahan-bahan pangan lokal.",
    "Pada zaman dahulu kala, di atas sebuah bukit kecil yang jauh dari pemukiman penduduk, di daerah Kalimantan Barat hiduplah seorang lelaki bernama Sambo.",
    "The Universal Declaration of Human Rights (UDHR) is an international document adopted by the United Nations General Assembly that enshrines the rights and freedoms of all human beings.",
    "Владимир Путин bermain tembak-tembakan bersama Pemimpin China bernama 习近平 dan Raja Arab, سلمان بن عبد العزیز آل سعود",
    "호복으로써 한복에 나타나는 몇 가지 특징이 있다. 기본적으로 활동성을 중시하며 딱 붙는 옷이 아니다."
]

for index,text in enumerate(text_examples):
  print("%d. "%(index+1),end="")
  print(tokenize(text),"\n")

1. ['mantan', 'ka', 'di', 'v', 'pro', 'pam', 'polri', 'ir', 'jen', 'fer', 'dy', 'sam', 'bo', 'resmi', 'diber', 'henti', 'akan', 'secara', 'tidak', 'hor', 'mat', 'sebagai', 'anggota', 'polri', '.'] 

2. ['real', 'madrid', 'berhasil', 'menjadi', 'kam', 'pi', 'un', 'liga', 'champion', 's', '20', '21', '-', '20', '22', 'setelah', 'menyu', 'dah', 'i', 'per', 'lawan', 'an', 'seng', 'it', 'dari', 'liverpool', '.'] 

3. ['pasar', 'k', 'ang', 'en', 'merupakan', 'juga', 'ajang', 'kre', 'ati', 'v', 'itas', 'dan', 'produk', 'tiv', 'itas', 'dalam', 'peng', 'olah', 'an', 'pangan', 'berbasis', 'nilai', '-n', 'ilai', 'lo', 'cal', 'dan', 'bahan', '-', 'bahan', 'pangan', 'lok', 'al.'] 

4. ['pada', 'zaman', 'dahulu', 'kala', ',', 'di', 'atas', 'sebuah', 'bukit', 'kecil', 'yang', 'jauh', 'dari', 'pemuki', 'man', 'penduduk', ',', 'di', 'daerah', 'kalimantan', 'barat', 'hidup', 'lah', 'seorang', 'lel', 'aki', 'bernama', 'sam', 'bo', '.'] 

5. ['the', 'uni', 'ver', 'sal', 'de', 'c', 'lar', 'ation', 'of', 'h

Dari pengamatan berdasar contoh dapat kita saksikan:

1. Kalimat berbahasa Indonesia berhasil ditokenisasi sebagaimana mestinya
2. Kalimat berbahasa Inggris atau asing yang menggunakan huruf latin tetap ditokenisasi dengan pola agak acak karena dataset yang digunakan berbeda bahasa.
3. Jika ada karakter yang tidak muncul pada daftar *token*, setiap karakter yang tidak diketahui pada kalimat dijadikan 1 token karena tidak di-*merge*.

# Referensi

1. https://wortschatz.uni-leipzig.de/en/download/Indonesian
2. https://huggingface.co/course/chapter6/5